# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
from numpy import mean
from numpy import absolute
from numpy import sqrt

from sklearn.linear_model import Perceptron
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score,mean_squared_error,r2_score

# Importing Datasets

In [3]:
train=pd.read_csv("train_dataset.csv")

In [4]:
test=pd.read_csv("test_dataset.csv")

# Extracting Data

In [5]:
train=train.set_index('Unnamed: 0')
X_train=train.iloc[:,0:9]
y_train=train.Survived

In [6]:
test=test.set_index('Unnamed: 0')
X_test=test.iloc[:,0:9]
y_test=test.Survived

# Taking a look at the data

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(534, 9)
(534,)
(179, 9)
(179,)


In [8]:
X_train.head()

,Pclass,Sex,Age,Fare,Embarked,FamilyCount,Title,Has Cabin,IsAlone
Unnamed: 0,,,,,,,,,
711,1,0,29.699118,26.5500,0,0,1,1,0
466,2,0,29.699118,0.0000,0,0,1,0,0
213,2,0,30.000000,13.0000,0,0,1,0,0
496,1,1,54.000000,78.2667,2,1,2,1,1
583,1,0,36.000000,40.1250,2,0,1,1,0


In [9]:
y_train.head()

Unnamed: 0
711    0
466    0
213    0
496    1
583    0
Name: Survived, dtype: int64

# Perceptron

In [10]:
perceptron = Perceptron(max_iter=200)
perceptron.fit(X_train, y_train)

y_pred = perceptron.predict(X_test)


In [11]:
acc_pr = accuracy_score(y_test,y_pred)
acc_pr

0.6368715083798883

# Hyperparameter Tuning to improve Accuracy

In [35]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid
max_iter = list(range(100,1100,100))

param_grid = {
    'eta0' : [0.0001, 0.001, 0.01, 0.1, 1.0],
    'max_iter' : max_iter
}
perceptron = Perceptron()

In [36]:
grid_search = GridSearchCV(estimator = perceptron, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


GridSearchCV(cv=3, estimator=Perceptron(), n_jobs=-1,
             param_grid={'eta0': [0.0001, 0.001, 0.01, 0.1, 1.0],
                         'max_iter': [100, 200, 300, 400, 500, 600, 700, 800,
                                      900, 1000]},
             verbose=2)

In [37]:
grid_search.best_params_

{'eta0': 0.0001, 'max_iter': 100}

In [38]:
grid_search.best_score_

0.6273408239700374

In [41]:
perceptron = grid_search.best_estimator_

# Cross validating the model

In [42]:
#define cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [43]:
scores = cross_val_score(perceptron, X_train, y_train, scoring='neg_mean_absolute_error',cv=cv, n_jobs=-1)

#view mean absolute error
mean(absolute(scores))

0.32012578616352205

In [44]:
#define cross-validation method to use
cv = KFold(n_splits=5, random_state=1, shuffle=True)

#use LOOCV to evaluate model
scores = cross_val_score(perceptron, X_train, y_train, scoring='neg_mean_squared_error',cv=cv, n_jobs=-1)

#view RMSE
sqrt(mean(absolute(scores)))

0.6533286310299243

Both MSE and RMSE values are not satisfactory

# Pickle the model

In [45]:
# save the model to disc
import pickle
filename = "Perceptron_model.pkl"
pickle.dump(perceptron,open(filename,'wb'))